In [ ]:
import pandas as pd
BASE_URL = "https://voyager.online/api/staking/validators"
page_size = 50

# Iterate over pages 1 to 3 (adjust the range as needed)
for p in range(1, 4):
    url = f"{BASE_URL}?search=&p={p}&ps={page_size}"
    data = requests.get(url).json()
    print(f"Page {p}:")
    print(data)


In [ ]:
host = "https://voyager.online/api/staking/validators?search=&p=1&ps=20"

result = requests.get(host).json()
result







In [4]:
import requests
import psycopg2

# PostgreSQL connection details - update these with your credentials
conn = psycopg2.connect(
    dbname="mydatabase",
    user="user",
    password="password",
    host="localhost",
    port="5432"  # typically 5432
)
cur = conn.cursor()

# Create the "validators" table if it doesn't exist, using proper numeric types
create_table_query = """
CREATE TABLE IF NOT EXISTS validators (
    address TEXT PRIMARY KEY,
    tokenAddress TEXT,
    isVerified BOOLEAN,
    totalStake NUMERIC,             -- changed from TEXT to NUMERIC
    totalStakePercentage NUMERIC,   -- changed from INTEGER to NUMERIC if decimals are needed
    ownStake NUMERIC,               -- changed from TEXT to NUMERIC
    delegatedStake NUMERIC,         -- changed from TEXT to NUMERIC
    totalDelegators INTEGER,
    revenueShare NUMERIC,           -- changed from INTEGER to NUMERIC if needed
    apr NUMERIC,                    -- changed from INTEGER to NUMERIC if decimals are needed
    rank TEXT,
    name TEXT,
    imgSrc TEXT,
    startTime BIGINT,
    poolAddress TEXT,
    stakerAddress TEXT
);
"""
cur.execute(create_table_query)
conn.commit()

BASE_URL = "https://voyager.online/api/staking/validators"
page_size = 100

# Iterate over pages 1, 2, and 3 (adjust the range as needed)
for p in range(1, 4):
    url = f"{BASE_URL}?search=&p={p}&ps={page_size}"
    response = requests.get(url)
    data = response.json()
    items = data.get("items", [])
    
    # Insert each item into the Postgres table
    for item in items:
        query = """
        INSERT INTO validators (
            address, tokenAddress, isVerified, totalStake, totalStakePercentage,
            ownStake, delegatedStake, totalDelegators, revenueShare, apr, rank,
            name, imgSrc, startTime, poolAddress, stakerAddress
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (address) DO NOTHING;
        """
        values = (
            item.get("address"),
            item.get("tokenAddress"),
            item.get("isVerified"),
            item.get("totalStake"),
            item.get("totalStakePercentage"),
            item.get("ownStake"),
            item.get("delegatedStake"),
            item.get("totalDelegators"),
            item.get("revenueShare"),
            item.get("apr"),
            item.get("rank"),
            item.get("name"),
            item.get("imgSrc"),
            item.get("startTime"),
            item.get("poolAddress"),
            item.get("stakerAddress")
        )
        cur.execute(query, values)
    
    conn.commit()
    print(f"Inserted data from page {p}")

cur.close()
conn.close()

Inserted data from page 1
Inserted data from page 2
Inserted data from page 3


In [3]:
import requests
import psycopg2

# PostgreSQL connection details - update these with your credentials
conn = psycopg2.connect(
    dbname="mydatabase",
    user="user",
    password="password",
    host="localhost",
    port="5432"  # typically 5432
)
cur = conn.cursor()

query_get_avg_delegated_stake_top_10 = """SELECT AVG(delegatedStake) 
FROM (
    SELECT delegatedStake 
    FROM validators 
    ORDER BY delegatedStake DESC
    limit 10
) subquery;"""


cur.execute(query_get_avg_delegated_stake_top_10)
result = cur.fetchall()
print(result[0][0] / 10**18)

cur.close()
conn.close()


15209196.437081446278959945


In [ ]:

# --- Configuration ---
RPC_URL = "https://free-rpc.nethermind.io"

# Replace these with your actual contract and address values.

import requests
from eth_hash.auto import keccak

def get_selector_from_name(function_name: str) -> str:
    """
    Compute the function selector for StarkNet by taking the Keccak hash
    of the function name and converting the result to a hexadecimal string.
    
    Note: StarkNet expects the selector as a hex string.
    """
    hash_bytes = keccak(function_name.encode("utf-8"))
    selector_int = int.from_bytes(hash_bytes, byteorder="big")
    return hex(selector_int)

# --- Configuration ---
RPC_URL = "https://free-rpc.nethermind.io/"

# Replace these placeholders with your actual addresses (as hex strings)
contract_address = "0x04718f5a0fc34cc1af16a1cdee98ffb20c31f5cd61d6ab07201858f4287c938d"  # StarkNet contract address
owner_address = "0x030016e7beABe590593a1958e31D2aFa2D06754Ea8468e3e720c781c3cA690d6"            # Owner's contract address
spender_address = "0x071c4334c5b387cbc84a3a62b16c6eb0481bf596054d18fd983e5f3fc27e489b"        # Spender's contract address

# --- Compute the entry point selector for "allowance" ---
allowance_selector = get_selector_from_name("allowance")
print("Computed selector for 'allowance':", allowance_selector)

# --- Build the JSON-RPC payload ---
payload = {
    "jsonrpc": "2.0",
    "method": "starknet_call",
    "params": [
        {
            "contract_address": contract_address,
            "entry_point_selector": allowance_selector,
            # Calldata: the function expects [owner, spender] as inputs.
            "calldata": [owner_address, spender_address]
        },
        "latest"  # Use the latest block
    ],
    "id": 1
}

# --- Make the RPC request ---
response = requests.post(RPC_URL, json=payload)
response.raise_for_status()  # Raise an exception if the call failed

# --- Parse the response ---
rpc_result = response.json()
# The RPC result is expected to be a list of hex strings; we assume the allowance is at index 0.
allowance_hex = rpc_result["result"][0]
allowance_dec = int(allowance_hex, 16)

# --- Output the results ---
print("Allowance (Hex):", allowance_hex)
print("Allowance (Decimal):", allowance_dec)



Computed selector for 'allowance': 0xc9e888a1026b19c8c0b57c72d63ed1737106aa10034105b980ba117bd0c29fe1


HTTPError: 404 Client Error: Not Found for url: https://free-rpc.nethermind.io/